In [1]:
# Connecting to Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing required libraries

In [2]:
import os
from openai import OpenAI
from google.colab import userdata

In [3]:
# Setting Groq client with API key(api key stored in colab secrets)
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key, base_url="https://api.groq.com/openai/v1")

### Task-1: Conversation management + summarization

#### 1. Maintain running conversation history

In [4]:
# List to store conversations
conversations = []

In [5]:
# Function to add user-assisstant conversation
def add_message(role, content):
  conversations.append({'role':role,'content':content})

# Function to get history of conversation
def get_history_text():
  chat_history = "\n".join(f"{m['role'].capitalize()}: {m['content']}" for m in conversations)
  return chat_history

#### 2. Summarization function

In [6]:
def summarize_with_groq(text):
  response = client.chat.completions.create(
      model="meta-llama/llama-4-maverick-17b-128e-instruct",
      messages = [
          {"role":"system", "content":"You are a helpful summarizer."},
          {"role":"user", "content":f"Summarize the follwing conversation in 50 words or less:\n{text}"}
      ],
      max_tokens=200
  )

  return response.choices[0].message.content

#### 3. Truncation Options

In [7]:
# Truncate by runs
def truncate_by_runs(n):
  return conversations[-n:]

# Truncate by characters
def truncate_by_chars(max_chars):
  text = get_history_text()
  return text[-max_chars:]

#### 4. Periodic summarization

In [8]:
run_num = 0 # counter for runs
k = 3 # sumarize after every 3 runs

In [9]:
# Function to summarize every k runs and replace history with summary
def periodic_summarization():
  global run_num, k, conversations
  run_num +=1
  print(f"Run:{run_num}")

  if run_num % k == 0:
    chat_history = get_history_text()
    summary = summarize_with_groq(chat_history)
    print("**Summarization triggered**")
    print("Summary:", summary)

    #Replacing full conversation with summary
    conversations = [{"role":"system","content":f"Summary so far: {summary}"}]

  else:
    print("**No summarization**")

#### 5. Demonstration

In [10]:
# Feed multiple conversation samples
add_message("user", "Hi, I'm Vamshi. Can you help me?")
add_message("content", "Sure - what do you need help with?")
periodic_summarization()

add_message("user", "I want to book a flight for next week.")
add_message("content", "Great, from which city are you flying?")
periodic_summarization()

add_message("user", "From Hyderabad.")
add_message("content", "Got it. And what is your destination?")
periodic_summarization()

add_message("user", "I’m going to Bangalore.")
add_message("content", "Okay. Do you have specific travel dates in mind?")
periodic_summarization()

add_message("user", "Yes, next Monday morning.")
add_message("content", "Perfect. Do you prefer economy or business class?")
periodic_summarization()

add_message("user", "Economy class is fine.")
add_message("content", "Alright, I’ll check flights for Hyderabad to Bangalore next Monday in economy.")
periodic_summarization()

Run:1
**No summarization**
Run:2
**No summarization**
Run:3
**Summarization triggered**
Summary: Vamshi requests help booking a flight from Hyderabad for next week, and the assistant asks for his destination.
Run:4
**No summarization**
Run:5
**No summarization**
Run:6
**Summarization triggered**
Summary: Vamshi requests help booking a flight from Hyderabad to Bangalore for next Monday. The assistant asks for travel dates and class preference, and Vamshi confirms next Monday morning and economy class. The assistant will now check available flights.
